# Price estimation

In [ ]:
%matplotlib widget

import sys

sys.path.append("../python")

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import ipywidgets.widgets as w
from mll1 import car_pricing

In [ ]:
import joblib
import json

reg = joblib.load("./cars_price_model.pkl")
with open("./config.json") as infile:
    config = json.load(infile)

In [ ]:
# The list of all brands
# Initialized with no value
brands = w.Dropdown(
    options=sorted(config["model_per_brand"].keys()), description="Brand: ", value=None
)

# The list of models for the current brand
# Initialized as empty and disabled until a brand is selected
models = w.Dropdown(description="Model: ", disabled=True)


# Update the model list when the brand is modified
def update_models(*args):
    # If a brand is selected
    # 1. Enable the models selector
    # 2. Set the list corresponding to the brand
    if brands.value is not None:
        models.disabled = False
        models.options = sorted(config["model_per_brand"][brands.value]) + ["Other"]
    else:  # Otherwise disable the models selector
        models.disabled = True


# The function will be triggered when
# the widget changes
brands.observe(update_models)


# Select all existing values for all models
def get_all_values(feat_name):
    return sorted(
        set(
            sum(
                (
                    list(model_feat[feat_name])
                    for model_feat in config["features_per_models"].values()
                ),
                [],
            )
        )
    ) + ["Other"]


all_sizes = get_all_values("engineSize")
all_fuel_types = get_all_values("fuelType")
all_transmissions = get_all_values("transmission")

# Initialize widets for all the other features
engineSizes = w.Dropdown(description="Engine Size: ", options=all_sizes)
fuelTypes = w.Dropdown(description="Fuel Type: ", options=all_fuel_types)
transmissions = w.Dropdown(description="Transmission: ", options=all_transmissions)
# For the year, use a slider
years = w.IntSlider(min=1970, max=2021, value=2021, description="Year")
# For the mileage, set the value manually
mileage = w.Text(value="10000", description="Mileage: ")

In [ ]:
from IPython.display import display, HTML
from ipywidgets import interact
import pandas as pd
import numpy as np


@interact(
    brand=brands,
    model=models,
    engine_size=engineSizes,
    fuel_type=fuelTypes,
    transmission=transmissions,
    year=years,
    mileage=mileage,
)
def compute_price(brand, model, engine_size, fuel_type, transmission, year, mileage):
    engine_size = np.nan if engine_size == "Other" else float(engine_size)
    mileage = 10000 if mileage == "" else float(mileage)

    if brand is None:
        ypred = "&nbsp;" * 10
    else:
        df = pd.DataFrame(
            [
                {
                    "brand": brand,
                    "model": model,
                    "engineSize": engine_size,
                    "fuelType": fuel_type,
                    "transmission": transmission,
                    "year": year,
                    "mileage": mileage,
                    "tax": np.nan,
                    "mpg": np.nan,
                }
            ]
        )[config["columns"]]

        ypred = f"{reg.predict(df)[0].clip(0, np.inf):0.0f}"

    display(
        HTML(
            f'Proposed price: <span style="font-size:1.5em; margin-top: 10px; background-color: lightgreen; padding: 5px">{ypred} \u00a3</span>'
        )
    )